In [21]:
####
#Clean Porn Dataset
#Tianhao
#2021/7/29
####

import pandas as pd
import datetime
import numpy as np
import xlsxwriter
import requests
import os

In [28]:
### Read Categories File
pornhub_categories = pd.read_excel(r'C:\Users\raymo\Documents\GitHub\Callander-Research\Porn database\Data\RAW\PH_gay\Categories.xlsx')

### Read xlsx File
local_path = 'C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/RAW/PH_gay/' #local path of where the raw data stored

store_path = 'C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/OUTPUT/' #local path of where the output data stored
file_names = ['2021-03-07.xlsx','2021-03-14.xlsx','2021-03-21.xlsx','2021-03-28.xlsx','2021-04-04.xlsx','2021-04-18.xlsx'
             ,'2021-05-16.xlsx','2021-05-30.xlsx','2021-06-13.xlsx','2021-06-20.xlsx','2021-07-18.xlsx','2021-07-25.xlsx'
             ] #file name

datasets = {}
for file in file_names:
    sample = pd.read_excel(local_path+file,engine=None) #read in files
    datasets[file] = sample



C:\Users\raymo\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [29]:
sample = pd.read_excel('C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/RAW/PH_gay/2021-06-20.xlsx',engine=None)

In [145]:
#Series types
#This function convert series into correct types and check any null

def convert_types(file_name, data):
    copy_data = data.copy()

    #extract the viewkeys from pornhub urls
    
    copy_data["url"] = [i.replace("https://www.pornhub.com/view_video.php?viewkey=","") for i in copy_data["url"]]
    
    #convert views(str) to int
    copy_data["views"] = copy_data["views"].fillna('0')
    copy_data["views"] = [int(str(i).replace(',' , '')) for i in copy_data["views"]]
    
    #convert approval to int
    copy_data["approval"] = copy_data["approval"].fillna('0')
    copy_data["approval"] = [int(str(i).replace('%',''))/100 for i in copy_data["approval"]]
    
    
    #convert upload_date to date_time
    extract_time_str = file_name.replace(".xlsx","").split("-")
    extract_time = datetime.date(int(extract_time_str[0]),int(extract_time_str[1]),int(extract_time_str[2]))
    upload_time = np.array([])
    for i in copy_data["upload_date"]:
        copy_data["upload_date"] = copy_data["upload_date"].fillna('0 years ago')
        temp = i.split(" ")
        if temp[0] == 'Yesterday':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= 1))
        elif temp[1] == 'week' or temp[1] == 'weeks':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*7))
        elif temp[1] == 'month' or temp[1] == 'months':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*30))
        elif temp[1] == 'year' or temp[1] == 'years':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*365))
        elif temp[1] == 'day' or temp[1] == 'days':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*1))
        else:
            upload_time = np.append(upload_time,'')
    copy_data["upload_date"] = upload_time
    
    #Convert all categories into string and check null
    copy_data["categories"] = [str(i) for i in copy_data["categories"]]
    
    #convert approval_pos and neg to int 
    copy_data["approval_pos"] = copy_data["approval_pos"].fillna('0')
    copy_data["approval_pos"] = [int(str(i).replace('%','').replace('K','000')) for i in copy_data["approval_pos"]]
    copy_data["approval_neg"] = copy_data["approval_neg"].fillna('0')
    copy_data["approval_neg"] = [int(str(i).replace('%','').replace(".0","").replace('K','000')) for i in copy_data["approval_neg"]]
    
    #Calculate True approval rate
    copy_data["approval"] = round(copy_data["approval_pos"]/(copy_data["approval_pos"]+copy_data["approval_neg"]),4)
    
    
    #Clean Actors Column
    copy_data["actors"] = copy_data["actors"].fillna('')
    copy_data["actors"] = [i.replace("Pornstars: ",'').replace("Suggest ",'').replace("\n ",'')
        .replace("Thank you for your suggestions! Our team is reviewing them!","") for i in copy_data["actors"]]
    copy_data["actors"] = ['NaN' if i == '' else i for i in copy_data["actors"]]

    #clean comment number
    copy_data["comments_number"] = copy_data["comments_number"].fillna('0')
    copy_data["comments_number"] = [int(i.replace('(','').replace(')','')) for i in copy_data["comments_number"]]
    
    #get bag of words for categories
    categories_data = modify_categories(copy_data)
    
    #Clean Comments
    comments = copy_data["comments_text"].fillna('')
    all_comments = []
    for c in comments:
        c_only = []
        temp = c.split(";")
        for i in temp:
            if '\n' not in i:
                c_only.append(i)
        all_comments.append(c_only)
    for i in np.arange(len(all_comments)):
        if all_comments[i] == [""]:
            all_comments[i] = []
    copy_data["comments_text"] = all_comments
    
    #create new table for comments
    comments_table = pd.DataFrame(columns = ['url','comments_text'])
    comments_table['url'] = copy_data["url"]
    comments_table['comments_text'] = copy_data["comments_text"]
    
    
    #clean related playlists videostill
    temp1 = []
    related_list = copy_data["related_videos_videostill"].fillna("")
    for i in related_list:
        temp1.append(i.split(';'))
    copy_data["related_videos_videostill"] = temp1
    
    #clean related playlists titles related_videos_videostill_alt
    temp2 = []
    related_list = copy_data["related_videos_videostill_alt"].fillna("")
    for i in related_list:
        temp2.append(i.split(';'))
    copy_data["related_videos_videostill_alt"] = temp2
    
    #create new table for related playlists
    related_videos = pd.DataFrame(columns = ['url','related_videos_videostill','related_videos_videostill_alt'])
    related_videos['url'] = copy_data['url']
    related_videos['related_videos_videostill'] = temp1
    related_videos['related_videos_videostill_alt'] = temp2
    
    #drop related_videos
    copy_data = copy_data.drop(columns = ["related_playlists","related_playlists_videostill_alt","related_playlists_videostill","related_videos_videostill","related_videos_videostill_alt",
                                          "related_videos","videostill_image_alt","categories","comments_text","comments_number"])
    
    return copy_data,categories_data,related_videos,comments_table


In [146]:
# This function create the bag of words for categories
def modify_categories(data):
    new_categories = [i.replace("Categories \n ","").replace(" \n Suggest","") for i in data["categories"]]
    bag = np.array([[]])
    for fullstring in data["categories"]: 
        temp = np.array([])
        for substring in pornhub_categories["Categories"]:
                
            if substring in fullstring:
                temp = np.append(temp, 1)
            elif substring not in fullstring:
                temp = np.append(temp, 0)
        
        bag = np.append(bag,temp)  
    bag = bag.reshape(data["categories"].size,pornhub_categories["Categories"].size)
    bag = pd.DataFrame(bag,index = data["url"] ,columns = pornhub_categories["Categories"])
    return bag



,url,comments_text
0,ph606c9b210339e,[ wow. easily one of the top 10 vids of all ti...
1,ph5db00da91be01,"[ Love your chubby body, love to be in between..."
2,ph60ace024ad1a7,[]
3,ph5f7f6e3e7df61,[]
4,ph5f2ca9248b63e,"[ great video,thanks]"
...,...,...
992,ph6086d8a953051,[ That video I enjoyed watching. It gets your ...
993,ph60153fccb8aae,[ Fuck take me from la 7 inch cock ????]
994,ph605311083e8f8,[ Hot! Is there more of this guy?]
995,ph5ee4d450bcbc2,"[ Wish i was there really hot, Super hot. Won..."


In [150]:
#export modified dataset

for k in datasets.keys():
    # k is the filename (date)
    modified = convert_types(k,datasets[k])
    #Create a Folder in the path
    new_path = store_path + k.replace(".xlsx","") + '/'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    
    #write out excel file
    modified[0].to_excel(new_path+"OUTPUT_" + k,sheet_name = 'Sheet 1')
    modified[1].to_excel(new_path+"OUTPUT_Categories_" + k,sheet_name = 'Sheet 1')
    modified[2].to_excel(new_path+"OUTPUT_Related_Videos_" + k,sheet_name = 'Sheet 1')
    modified[3].to_excel(new_path+"OUTPUT_Comments_" + k,sheet_name = 'Sheet 1')
    print("Output for file: ", k, " is successful :) ")

Output for file:  2021-03-07.xlsx  is successful :) 
Output for file:  2021-03-14.xlsx  is successful :) 
Output for file:  2021-03-21.xlsx  is successful :) 
Output for file:  2021-03-28.xlsx  is successful :) 
Output for file:  2021-04-04.xlsx  is successful :) 
Output for file:  2021-04-18.xlsx  is successful :) 
Output for file:  2021-05-16.xlsx  is successful :) 
Output for file:  2021-05-30.xlsx  is successful :) 
Output for file:  2021-06-13.xlsx  is successful :) 
Output for file:  2021-06-20.xlsx  is successful :) 
Output for file:  2021-07-18.xlsx  is successful :) 
Output for file:  2021-07-25.xlsx  is successful :) 


In [141]:
#This function automaticall download video_stillimages 
#Parameters:
#            path: the path of which the picture be store
#            data: the whole dataset
        
def helper(filename,store_path,data):
    new_path = store_path + filename.replace(".xlsx","") + '/video_stil_images/'
    url_keys = convert_types(filename,data)[0]["url"]
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    urls = data["videostill_image"]
    for i in np.arange(len(urls)):
        if not pd.isna(urls[i]):
            r = requests.get(urls[i], allow_redirects=True,stream = True)
            open(new_path+ url_keys[i] + '.jpg', 'wb').write(r.content)


In [ ]:
#Download all images
for k in datasets.keys():
    helper(k,store_path,datasets[k])